# Лекция 9.
# Исключения и менеджеры контекста

## Исключения
- Зачем нужны исключения
- Иерархия исключений в python
- Обработка исключения
- Интерфейс исключений
- Пользовательские исключения
- Оператор raise, цепочки исключения

### Зачем нужны исключения?
Исключения нужны для исключительных ситуаций, это ошибки, которые можно обрабатывать. <br/>
Например...

In [1]:
[0] * int(1e16)

MemoryError: 

In [2]:
import foobar

ModuleNotFoundError: No module named 'foobar'

In [3]:
[1, 2, 3] + 4

TypeError: can only concatenate list (not "int") to list

### Встроенные исключения
https://docs.python.org/3/library/exceptions.html#exception-hierarchy

#### BaseException
Базовый класс для встроенных исключений в Python.
Напрямую от класса BaseException наследуются только системные исключения и исключения, приводящие к завершению работы интерпретатора. <br/>
Все остальные встроенные исключения, а также  исключения, объявленные пользователем, должны наследоваться от класса _Exception_.

In [4]:
BaseException.__subclasses__()

[Exception, GeneratorExit, SystemExit, KeyboardInterrupt]

#### AssertionError
Возникает, когда условие оператора _assert_ не выполняется. <br/>
Оператор _assert_ используется для ошибок, которые могут возникнуть только в результате ошибки программиста, поэтому перехватывать __AssertionError__ считается дурным тоном.

In [5]:
assert 2 + 2 == 5, "Math still works!!!"

AssertionError: Math still works!!!

#### ImportError

In [6]:
import foobar

ModuleNotFoundError: No module named 'foobar'

#### NameError

In [7]:
foobar

NameError: name 'foobar' is not defined

#### AttributeError
Исключение _AttributeError_ поднимается при попытке прочитать или (в случае \_\_slots\_\_) записать значение в несуществующий атрибут.

In [8]:
object().foobar

AttributeError: 'object' object has no attribute 'foobar'

#### LookupError
Исключения _KeyError_ и _IndexError_ наследуются от базового класса _LookupError_ и возникают, если в контейнере нет элемента по указанному ключу или индексу.

In [9]:
{}['foobar']

KeyError: 'foobar'

In [10]:
[][0]

IndexError: list index out of range

#### ValueError
Используется в случаях, когда другие более информативные исключения, например, *KeyError*, неприменимы.

In [11]:
'foobar'.split('')

ValueError: empty separator

#### TypeError
Возникает, если оператор, функция или метод вызываются с аргументом несоответствующего типа.

In [12]:
b'foo' + 'bar'

TypeError: can't concat str to bytes

### Обработка исключений

#### try...except 
Для обработки исключений в Python используются операторы __try__ и __except__.
Ветка __except__ принимает два аргумента:
    1. выражение, возвращающее тип или кортеж типов,
    2. опциональное имя для перехваченного исключения.
Исключение _e_ обрабатывается веткой __except__, если её первый аргумент _expr_ можно сопоставить с исключением: _isinstance(e, expr)_. <br/>
При наличии нескольких веток __except__ интерпретатор сверху вниз ищет подходящую. 

In [ ]:
try:
    1/0
except (ValueError, ArithmeticError):
    pass
except TypeError as e:
    pass
except Exception as e: 
    pass

На месте выражения в ветке __except__ может стоять любое выражение, например, вызов функции или обращение к переменной.

In [ ]:
try:
    something_dangerous()
except Exception as e:
    try:
        something_else()
    except type(e): # Какое исключение мы перехватим?
        pass

Время жизни переменной _e_ ограничивается веткой __except__.

In [14]:
error = None
try:
    1 + "42"
except TypeError as e:
    error = e
    pass # Что делать, если нам нужно e?

error

TypeError("unsupported operand type(s) for +: 'int' and 'str'")

#### try...finally
Иногда требуется выполнить какое-то действие вне зависимости от того, произошло исключение или нет, например, закрыть файл, сетевое соединение, примитив синхронизации, etc...

In [15]:
handle = open("example.txt", "wt")
try:
    print(f'{handle.name} closed? {handle.closed}')
    raise
finally:
    handle.close()

example.txt closed? False


RuntimeError: No active exception to reraise

In [16]:
print(f'{handle.name} closed? {handle.closed}')

example.txt closed? True


#### try...else
С помощью ветки _else_ можно выполнить какое-то действие в ситуации, когда внутри _try_ блока не возникло исключения.

In [17]:
try:
    handle = open('example.txt', 'wt')
except IOError as e:
    print(e, file=sys.stderr)
else:
    print('Success open')
    handle.close()

Success open


In [ ]:
# Чему лучше такого варианта?
try:
    handle = open('example.txt', 'wt')
    print('Success open')
    handle.close()
except IOError as e:
    print(e, file=sys.stderr)

#### Полная форма

In [ ]:
try:
    1/0
except Exception :
    print('ImportError occured')
except ZeroDivisionError:
    print('It\'s Zero Division!')
except ValueError:
    print('Maybe ValueError??')
else:
    print('No exception at all!!!')
finally:
    print('Bye')

### Исключения, объявленные пользователем
Для объявления нового типа исключения достаточно объявить класс, наследующийся от базового класса __Exception__. <br/>
Хорошая практика при написании библиотек на Python — объявлять свой базовый класс исключений.

In [18]:
class BaseLibraryException(Exception):
    pass

In [19]:
class SpecificException(BaseLibraryException):
    def __str__(self):
        return 'My custom exception'

In [20]:
try:
    raise SpecificException
except BaseLibraryException as e:
    print(f'Caught `{e}`')

Caught `My custom exception`


### Интерфейс исключений
- атрибут __args__ хранит кортеж аргументов, переданных конструктору исключения, 
- атрибут __\_\_traceback\_\___ содержит информацию о стеке вызовов на момент возникновения исключения.

In [21]:
try:
    1 + "42"
except Exception as e:
    caught = e

In [22]:
caught.args

("unsupported operand type(s) for +: 'int' and 'str'",)

In [23]:
caught.__traceback__

In [24]:
import traceback
traceback.print_tb(caught.__traceback__)

  File "<ipython-input-21-00129d968d0d>", line 2, in <module>
    1 + "42"


### Оператор raise

In [25]:
raise TypeError('type mismatch')

TypeError: type mismatch

Если вызвать оператор _raise_ без аргумента, то он поднимет последнее пойманное исключение.

In [26]:
try:
    1 / 0
except Exception:
    print('Caught something!')
    raise 

Caught something!


ZeroDivisionError: division by zero

Если такого исключения нет, то __RuntimeError__

In [27]:
raise

RuntimeError: No active exception to reraise

#### Оператор raise from

In [34]:
error = None
try:
    {}["foobar"]
except KeyError as e:
#     try:
    raise KeyError("Ooops!") from e
#     except Exception as e1:
#         error = e1

# error.args

KeyError: 'Ooops!'

#### Цепочки исключений

In [30]:
try:
    {}['foobar']
except KeyError:
    'foobar'.split('')

ValueError: empty separator

### Исключения - резюме
- Механизм обработки исключений в Python похожна аналогичные конструкции в С++ и Java, но Python расширяет привычную пару __try … except__ веткой __else__.
- Поднять исключение можно с помощью оператора __raise__, его семантика эквивалентна throw в C++ и Java.
- В Python много встроенных типов исключений, которые можно и нужно использовать при написании функций и методов: https://docs.python.org/3/library/exceptions.html
- Для объявления нового типа исключения достаточно унаследоваться от базового класса __Exception__.
- Два важных правила при работе с исключениями: 
    1. минимизируйте размер ветки try,
    2. всегда старайтесь использовать наиболее специфичный тип исключения в ветке except.

### Задача 1
Даны два значения: a, b. Необходимо выполнить деление a/b и вывести ответ. <br/>
Все взаимодействие происходит через __input__ пользователя. <br/>
Пользователь вводит сначала число пар которые он будет вводить. Затем вводит все пары через пробел. 
Пример взаимодействия: <br/>
`3
1 0
Error code: division by zero
9 $
Error code: invalid literal for int() with base 10: '$'
6 2
3`

## Менеджеры контекста
- Зачем нужны менеджеры контекста
- Протокол менеджеров контекста
- Модуль _contextlib_

### Зачем нужны менеджеры контекста?

In [35]:
# Без контекстных менеджеров
out = open('output.txt', 'w')

for i in range(10000):
    if i == 5000:
        raise
    print(f'Line number {i}', file=out)
    
out.close()

RuntimeError: No active exception to reraise

In [36]:
!tail -n 3 ./output.txt

Line number 4617
Line number 4618
Line number 4619


In [37]:
# С контекстными менеджерами
with open('output1.txt', 'w') as out:
    for i in range(10000):
        if i == 5000:
            raise
        print(f'Line number {i}', file=out)

RuntimeError: No active exception to reraise

In [38]:
!tail -n 3 output1.txt

Line number 4997
Line number 4998
Line number 4999


In [ ]:
!rm output.txt output1.txt

### Протокол менеджеров контекста
Протокол менеджеров контекста состоит из двух методов.
1. Метод __\_\_enter\_\___ инициализирует контекст, например, открывает файл или захватывает мьютекс. Значение,
возвращаемое этим методом, записывается по имени, указанному после оператора __as__.
2. Метод __\_\_exit\_\___ вызывается после выполнения тела оператора __with__. Метод принимает три аргумента:
    - тип исключения,
    - само исключение и
    - объект типа traceback. 
    
Если в процессе исполнения тела оператора with было поднятно исключение, метод __\_\_exit\_\___ может подавить его, вернув _True_.
Экземпляр любого класса, реализующего эти два метода, является менеджером контекста.

In [40]:
from functools import partial

class CustomOpen:
    def __init__(self, path, *args, **kwargs):
        self.path = path
        self.open_func = partial(open, path, *args, **kwargs)
        print(f'Init CM for {path}')
        
    def __enter__(self):
        self.handler = self.open_func()
        print(f'Opened {self.handler.name}')
        return self.handler
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.handler.close()
        print(f'Closed {self.handler.name}')
        del self.handler

In [45]:
with CustomOpen('./example.txt', 'w') as f1:
    a = 'ksjdf'
    f1.write('Hello World!!!')

# print(f'{f1.name} closed? {f1.closed}')
a

Init CM for ./example.txt
Opened ./example.txt
Closed ./example.txt


'ksjdf'

In [42]:
!cat example.txt

Hello World!!!


### Процесс исполнения оператора __with__

In [ ]:
with CustomOpen('./example.txt', 'w') as f:
    f.write('Hello World!!!')

In [46]:
manager = CustomOpen('./example.txt', 'w')
f = manager.__enter__()

try:
    raise 
except Exception:
    pass

try:
    f.write('Hello World!!!')
finally:
    import sys
    exc_type, exc_value, tb = sys.exc_info()
    print(exc_value)
    suppress = manager.__exit__(exc_type, exc_value, tb)
    if exc_value is not None and not suppress:
        raise exc_value

Init CM for ./example.txt
Opened ./example.txt
None
Closed ./example.txt


### Вложенные менеджеры контекста

In [48]:
with CustomOpen('./example1.txt', 'w'), \
      CustomOpen('./example2.txt', 'w'):
    print('Inside managers')

Init CM for ./example1.txt
Opened ./example1.txt
Init CM for ./example2.txt
Opened ./example2.txt
Inside managers
Closed ./example2.txt
Closed ./example1.txt


In [49]:
with CustomOpen('./example3.txt', 'w') as f1:
    with CustomOpen('./example4.txt', 'w') as f2:
        print('Inside managers')

Init CM for ./example3.txt
Opened ./example3.txt
Init CM for ./example4.txt
Opened ./example4.txt
Inside managers
Closed ./example4.txt
Closed ./example3.txt


### Примеры
- open
- tempfile
- threading.Lock

### Задача 2
Написать контекстный менеджер __cd__, который меняет текущую директорию на заданную.  
При входе в контекст нужно запомнить прежнюю директорию и при выходе восстановить ее.

### Модуль contextlib
https://docs.python.org/3/library/contextlib.html

#### contextlib.contextmanager

In [54]:
import contextlib

@contextlib.contextmanager
def CustomOpenCL(path, *args, **kwargs):
    f = open(path, *args, **kwargs)
    
    try:
        print('Enter context')
        yield f
    finally:
        print('Exit context')
        f.close()

        
with CustomOpenCL('./more_example.txt', 'w'):
    print('Inside context')

Enter context
Inside context
Exit context


#### contextlib.ContextDecorator

In [51]:
import contextlib
class MyContextDecorator(contextlib.ContextDecorator):
    def __init__(self, no):
        self.no = no
        
    def who_am_i(self):
        return f'I\'m number {self.no}'
        
    def __enter__(self): 
        print(f'Before #{self.no}')
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f'After #{self.no}')

In [52]:
@CustomOpenCL(54)
def managed_func():
    print('Managed code')

In [53]:
managed_func()

Before #54
Managed code
After #54


#### Встроенные contextlib менеджеры

In [55]:
with contextlib.redirect_stdout(open('out.txt', 'w')):
    help(pow)

In [56]:
!cat out.txt

Help on built-in function pow in module builtins:

pow(x, y, z=None, /)
    Equivalent to x**y (with two arguments) or x**y % z (with three arguments)
    
    Some types, such as ints, are able to use a more efficient algorithm when
    invoked using the three argument form.



In [57]:
from urllib.request import urlopen

with contextlib.closing(urlopen('https://www.python.org')) as page:
    print(page.info())

Connection: close
Content-Length: 48843
Server: nginx
Content-Type: text/html; charset=utf-8
X-Frame-Options: DENY
Via: 1.1 vegur
Via: 1.1 varnish
Accept-Ranges: bytes
Date: Mon, 15 Jun 2020 17:10:27 GMT
Via: 1.1 varnish
Age: 899
X-Served-By: cache-bwi5150-BWI, cache-lax8647-LAX
X-Cache: HIT, HIT
X-Cache-Hits: 1, 2
X-Timer: S1592241028.819687,VS0,VE0
Vary: Cookie
Strict-Transport-Security: max-age=63072000; includeSubDomains




In [58]:
import os

with contextlib.suppress(FileNotFoundError):
    os.remove('somefile.tmp')

### Менеджеры контекста - резюме
- Менеджеры контекста - важный и удобный паттерн при работе с ресурсами, которые нужно правильно освобождать
- Менеджером контекста является любой объект, который реализует 2 метода: __\_\_enter\_\___ и __\_\_exit\_\___
- В стандартной библиотеке Python а также в сторонних библиотеках много менеджеров контекста
- Модуль __contextlib__ стандартной библиотеки делает создание своих менеджеров простым и приятным, а также содержит много готовых менеджеров

## Домашнее задание

### Задание 1
Реализовать контекстный менеджер - аналог __tempdir__. 
1. При входе в контекст создается директория с уникальным именем. 
2. Вся дальнейшая работа ведется в этой директории (она становится текущей).
3. При выходе из контекста директория удаляется вместе со всеми файлами в ней. 
4. Рабочей директорией становиться та, что была до входа в контекст.

Использовать протокол менеджеров контекста (реализовать методы `__enter__` и `__exit__`).  
Продемонстрировать работу своего менеджера: пока находимся в его контексте, пишем что-нибудь на диск, после выхода - проверяем, что все подчистилось без каких-то дополнительных команд.

### Задание 2
Реализовать контекстный менеджер, выводящий в файл следующую информацию:
- дата
- время выполнения кода
- информация о возникшей ошибке (в коде, обернутом контекстным менеджером).  

Файл указать при конструировании менеджера.  
Файл открывается в режиме `append`, чтобы при вызове менеджера с одним и тем же файлом информация дописывалась (такой самописный лог).  
Выше ошибка прокидывается (происходит reraise).  
Используйте ContextDecorator для решения.

### Задание 3*
Пересмотрите задания, которые вы выполняли ранее в курсе, свои пет-проекты, курсовые и т.п.  
Найдите фрагменты, которые вы бы теперь переписали с использованием менеджеров контекста (если раньше их не использовали)?  
Переписывать код не нужно, достаточно объяснить преподавателю, почему контекстные менеджеры здесь - то, что нужно.  
Если вы и раньше использовали их в своем коде, тоже покажите!